The dataset includes 12 months of data of the following tables:

1. subscription - holds the current state per each subscription 
1. order and order_item - an order and the line items for each order 
1. products - our sku table 
1. event_log - think of it as an activity log which tracks actions taken thru out the lifecycle of a subscription 
1. event_type - reference table for type_ID field (event_log table) 
 
Here are some questions we would like you to think about, some were deliberately left open ended to open the room for your data creativity and curiosity.  
1. Categorize the data set 
1. Can you make any hypothesis around subscription cancellations predictors 
1. We drive value for our clients by 
  1. increasing subscription conversion, 
  1. send as many shipments per subscription or 
  1. use the IU (impulse upsell) feature to have customers add one time items to their next order, 
based on that how else can you use this data to impact the above 
1. Which additional datasets would you like to see to further investigate the above? What new problems would you hope to solve or insights gleaned with this new data?


ideas for 2:

1. recidivism
1. default frequency not adjusted
1. gifts (shipping name/address != billing)
1. cheaper elsewhere (competitive intel)
1. transition to another product (serial products; diapers at size 1 for x months)

Ideas for 3:
1. 3.A - eliminate barriers (unnecessary clicks etc) between product page and successful subscription; targeted incentives; eliminate fear (this is why amazon dash is good - timing is always right); product rotations (subscribe to a group of products not all shipped at once)
1. 3.B - best default frequency; subscription mgmt ease (in reminder email)?; gift detection (how to handle these?)
1. 3.C - serial products (sizes of diapers; toddler products after baby products etc); linked products with 'surprise' items (candy bars etc) but need to be sensitive (no candy bars with diet stuff).

Ideas for 4:
1. non-subscription basket data from clients.
1. clickstream and browser data
1. demographic data (facebook: https://www.washingtonpost.com/news/the-intersect/wp/2016/08/19/98-personal-data-points-that-facebook-uses-to-target-ads-to-you/)
1. merchant demographic data
1. product pricing data
1. OG data
  1. Customer 
  1. Merchant
  1. Payment
  1. Offers

categorize the dataset:
cluster customers according to behaviors?
cluster items according to behaviors?
find clusters in the item-customer graph

predicting upsell opportunities (lower incentives; different products)
other datasets: clickstream data - categorize users by activity before they arrive at product page

first things first, get a clear definition of cancellation
basic stats on cancellation
hypothesize factors associated with cancellations - activities, engagement level/factors, product specific factors (aging out - e.g. diaper sizes)

easiest thing to do is subsample, choose from 1 store to limit noise from inventory factors - maybe even limit to 1 item at 1 store to start? -- group_by itemid, then perform some within-group analysis?

2. DQ issues. I probably should've asked for the mysql dump instead but I've gotten things mostly working now. However, compare the results from running wc on each file vs select count(*) from the resulting table:

```
     wc   select     diff
8829339  8457167  372,172  ./event_log_nulls.csv
     82       82        0  ./event_type_nulls.csv
6725109  6725109        0  ./order_item_nulls.csv
5396104  5396040       64  ./order_order_nulls.csv
6840406  6840385       21  ./product_product_nulls.csv
1938553  1935275     3278  ./subscription_subscription_nulls.csv
```

In the cast of the event logs, I think that a bunch of that error is accounted for by emails embedded in the email_html field which often span many lines in the raw data. Compare the following in which we grep for lines starting the right way:

    >> grep ^\"[0-9] event_log_nulls.csv | wc
    8457772 51181676 1149669198

This leaves a discrepancy of 603 rows.

I'd like to just forget about these entries since they are relatively rare in all tables. Unfortunately, we know that in the event_log table a lot of those will be errors resulting from the difficulty parsing the email html so the missing rows will not be a random sample and something similar could be happening in the other tables.

data problems / questions:

1. is there a data dictionary?
1. email data problems

Idea:
flow chart of subscriptions
Need:
nodes: (event_counter, event_type)
edges: num_people between each

No way to isolate individual subscriptions within event_log???

Customer id 0?

`offer_id` int(11) DEFAULT NULL, -- promo data?
`one_time` tinyint(1) NOT NULL,

only way to get incentives data: discount_total/sub_total in orders table??

timestamps are CST - OG server time.

### Cancellation Modeling questions
---


is cancellation a repeat-offender situation?

does non-modal shipping frequency impact likelihood of cancellation?

cancellation as a function of engagement? number of shipments as a function of engagement.
how does method-of-engagement affect this? autoship?

explore diaper and size stuff? look at cancel_reasons, that is a lot of info. "gift" in cancellation_reason -- maybe billing_name != shipping_name?

subscription, product, merchant factors?

rate of tactical one-and-done subscription (TOaDS) for each merchant / product? (Tactical = using subs as tool to get discount)

what factors are in common for the above? display list of ranked "worst for ODS" products and merchants

workflow
---

Build list of subscriptions with an TOaDS


elect oi.order_id, oi.subscription_id, o.merchant_id, o.customer_id, (o.sub_total+o.tax_total+o.shipping_total+o.discount_total) as total, o.discount_total,

total = (sub_total + shipping_total + discount_total)

subscription - items table:

    create table subsc_item_orders select subscription_id, product_id, count(id) as num_orders from order_items where subscription_id is not NULL and product_id is not NULL group by subscription_id, product_id;

```
MariaDB [og]> select max(logged), min(logged) from event_log where logged > 0000;
+---------------------+---------------------+
| max(logged)         | min(logged)         |
+---------------------+---------------------+
| 2059-03-20 01:00:00 | 2015-08-17 00:00:46 |
+---------------------+---------------------+
1 row in set (0.00 sec)

MariaDB [og]> select count(logged) from event_log where logged > "2016-08-20 00:00:00";
+---------------+
| count(logged) |
+---------------+
|            17 |
+---------------+
1 row in set (0.00 sec)
```

Things to build
===

TOaDS definition
---
Cancelled subscription with only 1 fulfilled order and time/opportunity to have more.

Note: only 10k subscriptions are associated with multiple items vs 1.2M for 1 item (after excluding NULLs of course).

rough TOaDS counts:

```
MariaDB [og]> select count(*) from subsc_item_orders where subscription_id < 500000 and num_orders=1;
+----------+
| count(*) |
+----------+
|    92663 |
+----------+
1 row in set (0.32 sec)

MariaDB [og]> select count(*) from subsc_item_orders where subscription_id < 500000 and num_orders!=1;
+----------+
| count(*) |
+----------+
|   277816 |
+----------+
1 row in set (0.32 sec)
```

about 1/3 TOaDS.

Recidivism for TOaDS
---
Define TOaDS and aggregate across customer_id
Find customers that
1. sign up in month 1
2. start with a TOaDS
3. have >= 2 subscriptions

At what rate is subscription #2 a TOaDS?

vs.

customers that 

1. sign up in month 1
2. start with a non-TOaDS
3. have >= 2 subscriptions

Workflow: 
1. Get all customer_ids associated with (>= or exactly) 2 subscriptions
2. Apply TOaDS definition to both
3. Build simple 

Products and Merchants suffering most from TOaDS
---
Just rank these and display, look for common factors

Predicting TOaDS
---
features:
product and merchant susceptibility
time
customer prior TOaDS

```
create table toads select s.id, s.created, s.customer_id, s.merchant_id, sio.num_orders, (sio.num_orders = 1) as TOaDS from subscriptions as s inner join subsc_item_orders as sio on s.id = sio.subscription_id;


create table subs_toads_2014 select x.customer_id, x.merchant_id, sum(x.TOaDS) as num_toads, count(*) as num_subs from  (select * from toads where created >= "2014-01-01 00:00:00" and created < "2015-01-01 00:00:00")x group by x.customer_id, x.merchant_id;

create table yoy_toads_comparison select t2014.*, t2015.merchant_id as merchant_id_2015, t2015.num_toads as num_toads_2015, t2015.num_subs as num_subs_2015 from subs_toads_2014 as t2014 inner join subs_toads_2015 as t2015 on t2014.customer_id = t2014.customer_id;
```

orders per sub
---

```
create table orders_per_sub select s.merchant_id, sio.product_id, sio.num_orders, count(*) from subscriptions as s inner join subsc_item_orders as sio on s.id=sio.subscription_id group by product_id, num_orders;
```

```
MariaDB [og]> select ns.*, p.name, price from (select product_id, count(*) as num_subs from subscriptions group by product_id order by num_subs desc limit 5) ns inner join products as p on p.id = ns.product_id;
+------------+----------+--------------------------------------------------------------------------+-------+
| product_id | num_subs | name                                                                     | price |
+------------+----------+--------------------------------------------------------------------------+-------+
|     868284 |    55078 | Menâ€™s ROGAINEÂ® Unscented Foam 4-Month Supply                          | 59.99 |
|    5119164 |    26284 | GEÂ® REFRIGERATOR WATER FILTER                                           | 49.99 |
|    5657542 |    18079 | GilletteÂ® FusionÂ® ProGlideÂ® Manual Razor Blade Refills, 4 Count       | 19.49 |
|    6161241 |    13306 | GilletteÂ® FusionÂ® ProShieldâ„¢ Razor Blade Refills, 4 Count            | 22.45 |
|    4718598 |     9686 | Deep Pore Cleansing Brush Head (Single)                                  | 27.00 |
+------------+----------+--------------------------------------------------------------------------+-------+
```

```
MariaDB [og]> select ns.*, p.name, price from (select product_id, count(*) as num_subs from subscriptions group by product_id order by num_subs desc limit 20) ns inner join products as p on p.id = ns.product_id;
+------------+----------+--------------------------------------------------------------------------+--------+
| product_id | num_subs | name                                                                     | price  |
+------------+----------+--------------------------------------------------------------------------+--------+
|     868284 |    55078 | Menâ€™s ROGAINEÂ® Unscented Foam 4-Month Supply                          |  59.99 |
|    5119164 |    26284 | GEÂ® REFRIGERATOR WATER FILTER                                           |  49.99 |
|    5657542 |    18079 | GilletteÂ® FusionÂ® ProGlideÂ® Manual Razor Blade Refills, 4 Count       |  19.49 |
|    6161241 |    13306 | GilletteÂ® FusionÂ® ProShieldâ„¢ Razor Blade Refills, 4 Count            |  22.45 |
|    4718598 |     9686 | Deep Pore Cleansing Brush Head (Single)                                  |  27.00 |
|       2155 |     9550 | Whirlpool 4396841 Refrigerator Filter                                    |  44.90 |
|    5657555 |     8786 | GilletteÂ® MACH3Â® Turbo Razor Blade Refills,  5 Count                   |  16.99 |
|    5526406 |     8652 | GEÂ® REFRIGERATOR WATER FILTER                                           |  54.99 |
|       1643 |     8502 | Maytag Puriclean II UKF8001 Filter 4 UKF8001AXX                          |  53.95 |
|       1656 |     8163 | Whirlpool PUR 4396710 Replaces Kenmore EDR3RXD1                          |  44.90 |
|    5119165 |     7673 | GEÂ® REFRIGERATOR WATER FILTER 3-PACK                                    | 134.75 |
|     868292 |     7554 | Womenâ€™s ROGAINEÂ® Topical Solution 4-Month Supply                      |  59.99 |
|    5199597 |     7240 | Hair Growth 3 Bottle Bundle                                              | 119.97 |
|    1108415 |     6135 | Ultra Facial Cream 4.2 fl. oz. Jar                                       |  47.50 |
|    1108357 |     6061 | Creamy Eye Treatment with Avocado 0.5 fl. oz. Jar                        |  29.00 |
|    1125811 |     5990 | Concentrate, Low-Cal Orange, (32 oz. Bottle), Individual                 |  20.99 |
|    5657550 |     5400 | GilletteÂ® FusionÂ® Manual Razor Blade Refills, 4 Count                  |  17.74 |
|     251433 |     5384 | Whirlpool W10295370 Refrigerator Filter FILTER1                          |  49.90 |
|    4718599 |     5345 | Deep Pore Cleansing Brush Head (TwinPack)                                |  44.00 |
|       3006 |     5215 | 1" Air Filters Merv 13 - Case of 6 Filters (Best)                        |  54.95 |
+------------+----------+--------------------------------------------------------------------------+--------+
```

```
select s.frequency_days, sio.num_orders, count(*) 
from subsc_item_orders as sio  
inner join  subscriptions as s 
on s.id = sio.subscription_id 
where sio.product_id = 1108415 
and s.created < "2015-01-01 00:00:00" 
group by s.frequency_days, sio.num_orders
INTO OUTFILE 'cream_freq_orders.csv' FIELDS TERMINATED BY ',' ENCLOSED BY '' LINES TERMINATED BY '\n';

SELECT s.frequency_days, sio.num_orders, count(*)
FROM subsc_item_orders AS sio
INNER JOIN subscriptions AS s
ON s.id = sio.subscription_id
WHERE sio.product_id = 1108415
AND s.created < "2015-01-01 00:00:00"
GROUP BY s.frequency_days, sio.num_orders
INTO OUTFILE 'cream_freq_orders.csv' FIELDS TERMINATED BY ',' ENCLOSED BY '' LINES TERMINATED BY '\n';
```